In [2]:
import urllib.request
import ssl
from openpyxl import Workbook
from openpyxl.styles import Alignment
 
from bs4 import BeautifulSoup

In [3]:
def fetch_url_content(url):
    try:
        # 创建 SSL 上下文，指定更灵活的配置
        ssl_context = ssl.create_default_context()
        ssl_context.set_ciphers('DEFAULT@SECLEVEL=1')  # 设置允许较弱的加密套件
 
        # 发送 HTTP 请求并获取响应
        with urllib.request.urlopen(url, context=ssl_context) as response:
            # 读取响应内容
            html_content = response.read().decode('utf-8')  # 假设使用 UTF-8 编码
            return html_content
    except urllib.error.URLError as e:
        # 处理 URL 错误
        print("URL Error:", e)
    except urllib.error.HTTPError as e:
        # 处理 HTTP 错误
        print("HTTP Error:", e)
    except Exception as e:
        # 处理其他异常
        print("Error:", e)
 
 
def remove_spaces_with_replace(input_string):
    return input_string.replace(" ", "")
 
 
def write_to_excel(data):
    # 创建一个新的工作簿
    wb = Workbook()
    ws = wb.active
 
    # 写入表头
    headers = ["标题", "报名方式", "报名截止日期", "报名状态", "发布时间"]
    for col, header in enumerate(headers, start=1):
        ws.cell(row=1, column=col, value=header)
        # 设置表头单元格居中对齐
        ws.cell(row=1, column=col).alignment = Alignment(horizontal='center', vertical='center')
 
    # 写入数据
    for row_idx, row_data in enumerate(data, start=2):
        for col_idx, cell_data in enumerate(row_data, start=1):
            ws.cell(row=row_idx, column=col_idx, value=cell_data)
 
    # 保存 Excel 文件
    filename = "data.xlsx"
    wb.save(filename)
    print(f"Excel 文件 '{filename}' 已保存。")
 
 
data_all = []
 
 
def earn_data(url_):
    html_content = fetch_url_content(url_)
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
 
        # 找到类名为 idx_zbgg_hx 的表格
        table = soup.find('table', class_='idx_zbgg_hx')
 
        if table:
            # 找到表格中所有行（<tr> 元素）
            trs = table.find_all('tr')
 
            # 遍历每个<tr>元素
            for tr in trs:
                # 获取<tr>元素下的所有<td>元素
                tds = tr.find_all('td')
                e_data = []
                # 遍历每个<td>元素，并获取其内容
                for td in tds:
                    content = td.get_text().strip()
                    # print(remove_spaces_with_replace(content))
                    e_data.append(content)
                if e_data:
                    data_all.append(e_data)
        else:
            print("Table with class idx_zbgg_hx not found.")
        # print(data_all)
 
 
# 测试爬虫
if __name__ == "__main__":
    num_all = ['1716623439754', '1716623814468', '1716623858932', '1716623911740', '1716623927364', '1716623943959',
               '1716623962482', '1716623980186', '1716624002988', '1716624019847', '1716624041015', '1716624056504',
               '1716624074168']
    for i in range(0, len(num_all)):
        url = 'https://bp.cfldcn.com/article!list.do?categoryCode=zbgg&request_time='+num_all[i]  # 要爬取的网页 URL
        earn_data(url)
    # 写入数据到 Excel 文件
    write_to_excel(data_all)

Excel 文件 'data.xlsx' 已保存。
